In [11]:
import numpy as np
from PIL import Image

In [144]:
images = ['0.png', '1.png', '2.png', '3.png', '4.png', '5.png', '6.png', '7.png', '8.png', '9.png']

In [52]:
# 0
for img in images:
    print(np.average(np.asarray(Image.open(img))[1600, 1380//2]) > 180)

# 1
for img in images:
    print(np.average(np.asarray(Image.open(img))[2278//3, 1380//3]) > 150)

# 2
for img in images:
    print(np.average(np.asarray(Image.open(img))[2278//3, 1380//2]) > 200)

# 3
for img in images:
    print(np.average(np.asarray(Image.open(img))[2278//3, 920]) > 170)

# 4
for img in images:
    print(np.average(np.asarray(Image.open(img))[2278//2, 1380//3]) > 160)
    
# 5
for img in images:
    print(np.average(np.asarray(Image.open(img))[2278//2, 1380//2]) > 190)

# 6
for img in images:
    print(np.average(np.asarray(Image.open(img))[2278//2, 920]) > 170)
    
# 7
for img in images:
    print(np.average(np.asarray(Image.open(img))[1400, 1380//3]) > 200)
    
# 8
for img in images:
    print(np.average(np.asarray(Image.open(img))[1400, 1380//2]) > 200)
    
# 9
for img in images:
    print(np.average(np.asarray(Image.open(img))[1400, 920]) > 170)

False
True
False
False
False
False
False
False
